# Real Estate Data Preprocessing Pipeline
Notebook for cleaning and preparing property data for RAG pipeline


In [104]:
# Import required libraries
import pandas as pd
import re
import os
from typing import Optional, List

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


In [105]:
JSON_FILE_PATH = "/content/19-august-12am_page1-ALL_property1-908.json"
OUTPUT_DIR = "ready_data"

try:
    df = pd.read_json(JSON_FILE_PATH)
    print("✓ JSON file loaded successfully!")
    print(f"   Original dataset shape: {df.shape}")
    print(f"   Columns found: {list(df.columns)}")
except FileNotFoundError:
    print(f"❌ File not found: {JSON_FILE_PATH}")
    print("Please upload your JSON file to Colab and update the path above")
except Exception as e:
    print(f"❌ Error loading file: {e}")

✓ JSON file loaded successfully!
   Original dataset shape: (908, 134)
   Columns found: ['page', 'scraped_at', 'source', 'property_id', 'property_url', 'listing_id', 'price', 'property_type', 'tenure', 'bedrooms', 'bathrooms', 'receptions', 'has_epc', 'has_floorplan', 'display_address', 'outcode', 'agent', 'chain_free', 'title', 'status', 'number_of_photos', 'number_of_floorplans', 'address', 'council_tax_band', 'ground_rent', 'room__m', 'about_property', 'nearest_schools', 'nearest_stations', 'latitude', 'longitude', 'nearest_stations_distances', 'epc_rating', 'size_sq_feet', 'price_per_sqft', 'room_lounge_m', 'room_dining_room_m', 'room_office_m', 'room_kitchen_m', 'room_bedroom_one_m', 'room_bedroom_two_m', 'room_bedroom_three_m', 'room_bedroom_four_m', 'room_family_bathroom_m', 'room_master_bedroom_m', 'room_shower_room_m', 'room_bathroom_m', 'room_en_suite_m', 'room_porch_m', 'room_entrance_hallway_m', 'room_diner_m', 'room_reception_room_one_m', 'room_reception_room_two_m', 'roo

In [106]:
# @title Step 2: Select Relevant Columns
# @markdown Since zoopla scraper scraps many irrelevent columns, we need to select one which actually beneficial for our rag pipeline these includes following columns , most import ones are adresses, property type, lat lang, nearest schools and stations

print("🔄 Step 2: Selecting relevant columns...")

# Define columns to keep (in order of importance)
columns_to_keep = [
    "property_id",
    "property_url",
    "price",
    "property_type",
    "tenure",
    "bedrooms",
    "bathrooms",
    "receptions",
    "outcode",
    "chain_free",
    "number_of_photos",
    "number_of_floorplans",
    "address",
    "council_tax_band",
    "ground_rent",
    "nearest_stations",
    "nearest_schools",
    "latitude",
    "longitude",
    "epc_rating",
    "size_sqft",
    "title",
    "agent",
    "about_property",
    # Crime data fields
    "crime_summary",
    "crime_data",
]

# Filter to only keep columns that exist in the DataFrame
existing_columns = [col for col in columns_to_keep if col in df.columns]
df_cleaned = df[existing_columns].copy()

print(f"✓ Selected {len(existing_columns)} relevant columns")
print(f"   Kept columns: {existing_columns}")

🔄 Step 2: Selecting relevant columns...
✓ Selected 23 relevant columns
   Kept columns: ['property_id', 'property_url', 'price', 'property_type', 'tenure', 'bedrooms', 'bathrooms', 'receptions', 'outcode', 'chain_free', 'number_of_photos', 'number_of_floorplans', 'address', 'council_tax_band', 'ground_rent', 'nearest_stations', 'nearest_schools', 'latitude', 'longitude', 'epc_rating', 'title', 'agent', 'about_property']


In [107]:
# @title Step 3: Rename Fields
# @markdown We applied better naming conventions to columns

print("🔄 Step 3: Renaming fields...")

# Field mapping for better naming
field_mapping = {
    "outcode": "postcode",
    "latitude": "lat",
    "longitude": "lng",
    "agent": "agent_name",
    "about_property": "description",
}

# Rename fields that exist
renamed_fields = []
for old_name, new_name in field_mapping.items():
    if old_name in df_cleaned.columns:
        df_cleaned = df_cleaned.rename(columns={old_name: new_name})
        renamed_fields.append(f"{old_name} → {new_name}")

if renamed_fields:
    print("✓ Fields renamed:")
    for rename in renamed_fields:
        print(f"   {rename}")
else:
    print("✓ No fields needed renaming")

🔄 Step 3: Renaming fields...
✓ Fields renamed:
   outcode → postcode
   latitude → lat
   longitude → lng
   agent → agent_name
   about_property → description


In [108]:
# @title Step 4: Handle Missing Values
# @markdown Missing data would over croud our RAG pipeline and we need to Identify and handle missing data

print("🔄 Step 4: Handling missing values...")

# Check for missing values initially
missing_counts = df_cleaned.isnull().sum()
missing_data = missing_counts[missing_counts > 0]

if not missing_data.empty:
    print("⚠️  Missing values found:")
    for col, count in missing_data.items():
        percentage = (count / len(df_cleaned)) * 100
        print(f"   {col}: {count} ({percentage:.1f}%)")
else:
    print("✓ No missing values found")

# Impute missing values with mode for categorical columns
categorical_cols = [
    "bathrooms",
    "receptions",
    "property_type",
    "council_tax_band",
    "tenure",
]

filled_columns = []
for col in categorical_cols:
    if col in df_cleaned.columns and df_cleaned[col].isnull().sum() > 0:
        mode_val = df_cleaned[col].mode()
        if not mode_val.empty:
            df_cleaned[col] = df_cleaned[col].fillna(mode_val[0])
            filled_columns.append(f"{col} (mode: {mode_val[0]})")

if filled_columns:
    print("✓ Filled missing values:")
    for fill in filled_columns:
        print(f"   {fill}")
else:
    print("✓ No categorical columns needed filling")

🔄 Step 4: Handling missing values...
⚠️  Missing values found:
   price: 4 (0.4%)
   property_type: 21 (2.3%)
   tenure: 48 (5.3%)
   bedrooms: 8 (0.9%)
   bathrooms: 11 (1.2%)
   receptions: 24 (2.6%)
   postcode: 4 (0.4%)
   chain_free: 4 (0.4%)
   number_of_photos: 4 (0.4%)
   number_of_floorplans: 28 (3.1%)
   address: 4 (0.4%)
   council_tax_band: 172 (18.9%)
   ground_rent: 564 (62.1%)
   nearest_stations: 4 (0.4%)
   nearest_schools: 79 (8.7%)
   lat: 4 (0.4%)
   lng: 4 (0.4%)
   epc_rating: 239 (26.3%)
   title: 4 (0.4%)
   agent_name: 4 (0.4%)
   description: 4 (0.4%)
✓ Filled missing values:
   bathrooms (mode: 1.0)
   receptions (mode: 1.0)
   property_type (mode: semi_detached)
   council_tax_band (mode: A)
   tenure (mode: freehold)


In [109]:
# @title Step 5: Convert Data Types
# @markdown Since we have planned to apply self retrievel in our RAG and that work best with the attributes that are properly data types so we need to convert columns to appropriate data types

print("🔄 Step 5: Converting data types...")

# Convert numerical columns (exclude price; we'll build price_int later)
numeric_cols = ["bathrooms", "bedrooms", "receptions", "size_sqft"]
converted_cols = []

for col in numeric_cols:
    if col in df_cleaned.columns:
        original_type = df_cleaned[col].dtype
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="coerce")
        converted_cols.append(f"{col}: {original_type} → {df_cleaned[col].dtype}")

if converted_cols:
    print("✓ Data types converted:")
    for conversion in converted_cols:
        print(f"   {conversion}")
else:
    print("✓ No numeric columns needed conversion")

🔄 Step 5: Converting data types...
✓ Data types converted:
   bathrooms: float64 → float64
   bedrooms: float64 → float64
   receptions: float64 → float64


In [110]:
# @title Step 6: Clean Text Data
# @markdown We'll use utility functions we craeted earlier to clean our data set which includes following columns which we know have white spaces etc
print("🔄 Step 6: Cleaning text data...")


def clean_text(text) -> str:
    """Clean text data by removing extra whitespace and newlines."""
    if isinstance(text, str):
        # Remove extra whitespace and newline characters
        text = re.sub(r"\s+", " ", text).strip()
        return text
    return text

# Clean text data (exclude URLs and coordinates)
text_cols = [
    "description",
    "address",
    "title",
    "agent_name",
    "nearest_stations",
    "nearest_schools",
    "epc_rating",
    "council_tax_band",
]

cleaned_text_cols = []
for col in text_cols:
    if col in df_cleaned.columns:
        # Show example before cleaning
        if not df_cleaned[col].dropna().empty:
            sample_before = df_cleaned[col].dropna().iloc[0][:50] + "..." if len(str(df_cleaned[col].dropna().iloc[0])) > 50 else str(df_cleaned[col].dropna().iloc[0])

        df_cleaned[col] = df_cleaned[col].apply(clean_text)
        cleaned_text_cols.append(col)

if cleaned_text_cols:
    print(f"✓ Cleaned {len(cleaned_text_cols)} text columns:")
    for col in cleaned_text_cols:
        print(f"   {col}")
else:
    print("✓ No text columns needed cleaning")

🔄 Step 6: Cleaning text data...
✓ Cleaned 8 text columns:
   description
   address
   title
   agent_name
   nearest_stations
   nearest_schools
   epc_rating
   council_tax_band


In [111]:
# @title Step 7: Create Helper Columns
# @markdown Create numeric helper columns for better processing becaue we know we'll self query rag and it works better with typed data

print("🔄 Step 7: Creating numeric helper columns...")

helper_cols_created = []

# Create numeric helper columns
if "bedrooms" in df_cleaned.columns:
    df_cleaned["bedrooms_int"] = df_cleaned["bedrooms"].astype("Int64")
    helper_cols_created.append("bedrooms_int")

if "bathrooms" in df_cleaned.columns:
    df_cleaned["bathrooms_int"] = df_cleaned["bathrooms"].astype("Int64")
    helper_cols_created.append("bathrooms_int")

if "receptions" in df_cleaned.columns:
    df_cleaned["receptions_int"] = df_cleaned["receptions"].astype("Int64")
    helper_cols_created.append("receptions_int")

if "price" in df_cleaned.columns:
    # Build integer-only price for downstream use and vector DB metadata filtering
    df_cleaned["price_int"] = (
        pd.to_numeric(df_cleaned["price"], errors="coerce").round(0).astype("Int64")
    )
    helper_cols_created.append("price_int")

if "size_sqft" in df_cleaned.columns:
    df_cleaned["size_sqft_num"] = df_cleaned["size_sqft"].astype("float64")
    helper_cols_created.append("size_sqft_num")

print(f"✓ Created {len(helper_cols_created)} helper columns:")
for col in helper_cols_created:
    print(f"   {col}")

# Keep only integer price; drop original and float helper if present
cols_to_drop = [c for c in ["price", "price_num"] if c in df_cleaned.columns]
if cols_to_drop:
    df_cleaned = df_cleaned.drop(columns=cols_to_drop)
    print(f"✓ Dropped original columns: {cols_to_drop}")

🔄 Step 7: Creating numeric helper columns...
✓ Created 4 helper columns:
   bedrooms_int
   bathrooms_int
   receptions_int
   price_int
✓ Dropped original columns: ['price']


In [112]:
# @title Step 8: Filter Incomplete Properties

print("🔄 Step 8: Filtering out incomplete properties...")

# Filter out properties where essential fields are null
initial_count = len(df_cleaned)

# Remove properties with missing address, price_int, or description
df_cleaned = df_cleaned.dropna(subset=["address", "price_int", "description"])

filtered_count = len(df_cleaned)
removed_count = initial_count - filtered_count

if removed_count > 0:
    print(f"⚠️  Removed {removed_count} incomplete properties")
    print(f"   (missing address, price_int, or description)")
    print(f"✓ Kept {filtered_count} complete properties")
else:
    print("✓ All properties have complete essential data")

🔄 Step 8: Filtering out incomplete properties...
⚠️  Removed 4 incomplete properties
   (missing address, price_int, or description)
✓ Kept 904 complete properties


In [113]:
# @title Step 9: Data Quality Summary
# @markdown Review the final cleaned dataset

print("📊 Step 9: Data Quality Summary")
print("=" * 50)

print(f"📈 Dataset Overview:")
print(f"   Final shape: {df_cleaned.shape}")
print(f"   Properties: {len(df_cleaned)}")
print(f"   Features: {len(df_cleaned.columns)}")

print(f"\n💰 Price Statistics:")
if "price_int" in df_cleaned.columns:
    price_stats = df_cleaned["price_int"].describe()
    print(f"   Min: £{price_stats['min']:,.0f}")
    print(f"   Max: £{price_stats['max']:,.0f}")
    print(f"   Mean: £{price_stats['mean']:,.0f}")
    print(f"   Median: £{price_stats['50%']:,.0f}")

print(f"\n🏠 Property Types:")
if "property_type" in df_cleaned.columns:
    prop_counts = df_cleaned["property_type"].value_counts()
    for prop_type, count in prop_counts.head().items():
        percentage = (count / len(df_cleaned)) * 100
        print(f"   {prop_type}: {count} ({percentage:.1f}%)")

print(f"\n🛏️  Bedroom Distribution:")
if "bedrooms_int" in df_cleaned.columns:
    bedroom_counts = df_cleaned["bedrooms_int"].value_counts().sort_index()
    for bedrooms, count in bedroom_counts.head().items():
        percentage = (count / len(df_cleaned)) * 100
        print(f"   {bedrooms} bed: {count} ({percentage:.1f}%)")

print(f"\n📍 Location Coverage:")
if "postcode" in df_cleaned.columns:
    postcode_count = df_cleaned["postcode"].nunique()
    print(f"   Unique postcodes: {postcode_count}")

# Check final data quality
final_missing = df_cleaned.isnull().sum()
final_missing_data = final_missing[final_missing > 0]

if not final_missing_data.empty:
    print(f"\n⚠️  Remaining missing values:")
    for col, count in final_missing_data.items():
        percentage = (count / len(df_cleaned)) * 100
        print(f"   {col}: {count} ({percentage:.1f}%)")
else:
    print(f"\n✓ No missing values in essential columns")

📊 Step 9: Data Quality Summary
📈 Dataset Overview:
   Final shape: (904, 26)
   Properties: 904
   Features: 26

💰 Price Statistics:
   Min: £35,000
   Max: £2,000,000
   Mean: £328,743
   Median: £270,000

🏠 Property Types:
   semi_detached: 286 (31.6%)
   detached: 186 (20.6%)
   flat: 155 (17.1%)
   terraced: 148 (16.4%)
   end_terrace: 27 (3.0%)

🛏️  Bedroom Distribution:
   1 bed: 57 (6.3%)
   2 bed: 257 (28.4%)
   3 bed: 340 (37.6%)
   4 bed: 175 (19.4%)
   5 bed: 52 (5.8%)

📍 Location Coverage:
   Unique postcodes: 87

⚠️  Remaining missing values:
   bedrooms: 4 (0.4%)
   number_of_floorplans: 24 (2.7%)
   ground_rent: 560 (61.9%)
   nearest_schools: 75 (8.3%)
   epc_rating: 235 (26.0%)
   bedrooms_int: 4 (0.4%)


In [114]:
# @title Step 10: Save Cleaned Data
# @markdown Export the cleaned dataset to CSV and JSON formats

print("🔄 Step 10: Saving cleaned data...")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Generate output filenames
base_name = os.path.splitext(os.path.basename(JSON_FILE_PATH))[0]
csv_path = os.path.join(OUTPUT_DIR, f"{base_name}_cleaned.csv")
json_path = os.path.join(OUTPUT_DIR, f"{base_name}_cleaned.json")

# Save files
try:
    df_cleaned.to_csv(csv_path, index=False)
    df_cleaned.to_json(json_path, indent=2, orient="records")

    print("✅ Cleaned data saved successfully!")
    print(f"   📄 CSV: {csv_path}")
    print(f"   📄 JSON: {json_path}")

    # Show file sizes
    csv_size = os.path.getsize(csv_path) / 1024 / 1024  # MB
    json_size = os.path.getsize(json_path) / 1024 / 1024  # MB

    print(f"   📊 CSV size: {csv_size:.1f} MB")
    print(f"   📊 JSON size: {json_size:.1f} MB")

except Exception as e:
    print(f"❌ Error saving files: {e}")

🔄 Step 10: Saving cleaned data...
✅ Cleaned data saved successfully!
   📄 CSV: ready_data/19-august-12am_page1-ALL_property1-908_cleaned.csv
   📄 JSON: ready_data/19-august-12am_page1-ALL_property1-908_cleaned.json
   📊 CSV size: 3.7 MB
   📊 JSON size: 4.2 MB


In [115]:
# @title Final Results
# @markdown Display final processing results

print("DATA PREPROCESSING COMPLETED!")

print(f"\n Processing Summary:")
print(f"   Original records: {df.shape[0] if 'df' in locals() else 'N/A'}")
print(f"   Final records: {len(df_cleaned)}")
print(f"   Features: {len(df_cleaned.columns)}")
print(f"   Data quality: {'High' if final_missing_data.empty else 'Good'}")

print(f"\n Output Files:")
print(f"   📄 {csv_path}")
print(f"   📄 {json_path}")

print(f"\n Next Steps:")
print("   1. Download the cleaned files from the output directory")
print("   2. Use the cleaned data for vector embedding")
print("   3. Set up your RAG pipeline with the processed data")
print("   4. Consider additional feature engineering if needed")

# Display sample of cleaned data
print(f"\n👀 Sample of Cleaned Data:")
print(df_cleaned.head(3))

DATA PREPROCESSING COMPLETED!

 Processing Summary:
   Original records: 908
   Final records: 904
   Features: 26
   Data quality: Good

 Output Files:
   📄 ready_data/19-august-12am_page1-ALL_property1-908_cleaned.csv
   📄 ready_data/19-august-12am_page1-ALL_property1-908_cleaned.json

 Next Steps:
   1. Download the cleaned files from the output directory
   2. Use the cleaned data for vector embedding
   3. Set up your RAG pipeline with the processed data
   4. Consider additional feature engineering if needed

👀 Sample of Cleaned Data:
   property_id                                       property_url  \
0     71084004  https://www.zoopla.co.uk/for-sale/details/7108...   
1     71082046  https://www.zoopla.co.uk/for-sale/details/7108...   
2     69999191  https://www.zoopla.co.uk/for-sale/details/6999...   

  property_type     tenure  bedrooms  bathrooms  receptions postcode  \
0      terraced  leasehold       2.0        1.0         1.0      BL1   
1          flat  leasehold      